In [1]:
import os 
import re
from konlpy.tag import Okt 
okt = Okt()

dirs = ['it-life-hack', 'movie-enter']

docterm = []
label = []
tmp1 = []
tmp2 = ''

for i, d in enumerate(dirs):
    files = os.listdir('./'+d)
    for file in files:
        f = open('./' + d + '/' + file, 'r', encoding = 'utf-8')
        text = f.read()
        
        reg_text = re.sub(r'[0-9a-zA-Z]+', '', text)
        reg_text = re.sub(r'[:;/+\.-]', '', reg_text)
        reg_text = re.sub(r'[\s\n]', '', reg_text)
        
        for token in okt.nouns(reg_text):
            tmp1.append(token)
            tmp2 = ' '.join(tmp1)
        docterm.append(tmp2)
        tmp1 = []
        
        label.append(i)
        f.close()

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd 

tv = TfidfVectorizer(min_df = 0.05, max_df = 0.5, sublinear_tf = True)
docterm_tv = tv.fit_transform(np.array(docterm))
docterm_tfidf = docterm_tv.toarray()

pd.DataFrame(docterm_tfidf).head()

,0,1,2,3,4,5,6,7,8,9,...,562,563,564,565,566,567,568,569,570,571
0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.07376,0.0,0.0,...,0.000000,0.0,0.0,0.084632,0.00000,0.0,0.0,0.098032,0.0,0.0
1,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.070737,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.087840,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.000000,0.100707,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.100707,0.0,0.0,0.000000,0.14957,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0


In [11]:
docterm_tfidf = pd.DataFrame(docterm_tfidf)

In [12]:
label = pd.DataFrame(label)
label = label.rename(columns={0:'label'})

docterm_df = pd.concat([docterm_tfidf, label], axis = 1)
docterm_df.head()

,0,1,2,3,4,5,6,7,8,9,...,563,564,565,566,567,568,569,570,571,label
0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.07376,0.0,0.0,...,0.0,0.0,0.084632,0.00000,0.0,0.0,0.098032,0.0,0.0,0
1,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0,0
2,0.0,0.070737,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0,0
3,0.0,0.000000,0.100707,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.000000,0.14957,0.0,0.0,0.000000,0.0,0.0,0
4,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0,0


In [13]:
from sklearn.metrics.pairwise import cosine_similarity

docterm_0 = docterm_df[docterm_df['label'] == 0]
docterm_0 = docterm_0.drop('label', axis = 1)

sim0 = cosine_similarity(docterm_0.T)
sim0_df = pd.DataFrame(sim0)
sim0_df

,0,1,2,3,4,5,6,7,8,9,...,562,563,564,565,566,567,568,569,570,571
0,1.000000,0.205519,0.153420,0.000000,0.022039,0.000000,0.000000,0.127624,0.000000,0.023377,...,0.000000,0.000000,0.000000,0.252665,0.044478,0.0,0.082310,0.000000,0.096540,0.000000
1,0.205519,1.000000,0.091144,0.000000,0.106671,0.150558,0.073067,0.294203,0.000000,0.193100,...,0.138849,0.000000,0.000000,0.216311,0.155980,0.0,0.237658,0.148796,0.164372,0.118615
2,0.153420,0.091144,1.000000,0.048929,0.000000,0.038366,0.065159,0.081113,0.000000,0.070562,...,0.024669,0.000000,0.000000,0.163622,0.107437,0.0,0.035241,0.024021,0.000000,0.052168
3,0.000000,0.000000,0.048929,1.000000,0.079386,0.064388,0.265024,0.088491,0.000000,0.000000,...,0.132830,0.060286,0.000000,0.073283,0.085702,0.0,0.000000,0.000000,0.000000,0.000000
4,0.022039,0.106671,0.000000,0.079386,1.000000,0.113064,0.000000,0.013833,0.000000,0.024613,...,0.000000,0.163831,0.000000,0.024088,0.146776,0.0,0.067580,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
568,0.082310,0.237658,0.035241,0.000000,0.067580,0.142428,0.000000,0.040181,0.000000,0.379419,...,0.000000,0.000000,0.000000,0.076405,0.047578,0.0,1.000000,0.056864,0.000000,0.000000
569,0.000000,0.148796,0.024021,0.000000,0.000000,0.275972,0.055598,0.115324,0.051015,0.045938,...,0.211807,0.079057,0.076104,0.236115,0.046965,0.0,0.056864,1.000000,0.000000,0.182260
570,0.096540,0.164372,0.000000,0.000000,0.000000,0.070661,0.000000,0.049852,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.024798,0.022664,0.0,0.000000,0.000000,1.000000,0.000000


In [14]:
sim0_stack = sim0_df.stack()

index = pd.Series(sim0_stack.index.values)
value = pd.Series(sim0_stack.values)

print(index.head())
print(value.head())

0    (0, 0)
1    (0, 1)
2    (0, 2)
3    (0, 3)
4    (0, 4)
dtype: object
0    1.000000
1    0.205519
2    0.153420
3    0.000000
4    0.022039
dtype: float64


In [19]:
tmp3 = []
tmp4 = []
for i in range(len(index)):
    if value[i] >= 0.5 and value[i] <= 0.9:
        tmp1 = str(index[i][0]) + ' ' + str(index[i][1])
        tmp2 = [int(s) for s in tmp1.split()]
        tmp3.append(tmp2)
        tmp4 = np.append(tmp4, value[i])
        
tmp3 = pd.DataFrame(tmp3)
tmp3 = tmp3.rename(columns = {0 : 'node1', 1 : 'node2'})
tmp4 = pd.DataFrame(tmp4)
tmp4 = tmp4.rename(columns = {0 : 'weight'})
sim0_list = pd.concat([tmp3, tmp4], axis = 1)

sim0_list.head()

,node1,node2,weight
0,0,315,0.522481
1,0,415,0.808384
2,1,123,0.527505
3,1,277,0.570043
4,2,130,0.802814
